# Agentic RAG

이번 챕터에서는 문서 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성하는 에이전트를 만들어 보겠습니다.

질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변하는 에이전트를 만들어 보겠습니다.

**참고**

- RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행한다면 이를 **Agentic RAG** 라고 부릅니다.

## 도구(Tools)

Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있습니다.

Tavily Search 는 그 중 대표적인 **검색 도구** 입니다. 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있습니다. 도구는 이처럼 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공합니다.


### 웹 검색도구: Tavily Search

LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아야 합니다.

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록 합니다.

`.env` 파일에 다음과 같이 등록합니다.

- `TAVILY_API_KEY=발급 받은 Tavily API KEY 입력`

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=3)

`search.invoke` 함수는 주어진 문자열에 대한 검색을 실행합니다.

`invoke()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행합니다.


In [3]:
# 검색 결과를 가져옵니다.
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

[{'title': '(ENG) 카카오프렌즈 판교아지트 전체둘러보기 춘식이 ... - YouTube',
  'url': 'https://www.youtube.com/watch?v=BpynjxkklPc&pp=ygUNI-q3gOyXveq1rOuniA%3D%3D',
  'content': '... 카카오프렌즈 판교아지트 ✓주소 : 경기 성남시 분당구 판교역로 166 1층 ✓전화번호 : 031-601-7225 ✓영업시간 : 평일 10시 ~ 21시 / 주말공휴일 12',
  'score': 0.93425745},
 {'title': '카카오프렌즈 판교 아지트점 판교의 카카오프렌즈를 소개합니다.',
  'url': 'https://m.blog.naver.com/hj961030/222923648353',
  'content': '춘식이를 좋아하시는 분들~!\n지금 놓지지 마시고 꼭꼭 춘식이 풍선 받으러 출발해보아요!\n\n\n\n작은 인테리어까지도 너무 작고 소중한\n카카오프렌즈 판교 아지트점!!\n\n\n선착순으로 받을 수 있는 춘식이 얼굴 풍선으로\n더욱 즐거운 시간을 만들어보면 어떨까요?\n\n\n\n\n\n포토존도 많아서 사진 찍기 좋아요-!!\n\n\n아참-!\n판교 아지트점에서 숨은 라이언 찾기 이벤트도 진행하고 있으니\n참여하면 좋겠죠?!!\n\n그럼 오늘은 여기까지 소개를 드리도록 하겠습니다!\n직접 보면 더 예쁜 카카오프렌즈 판교 아지트점\n\u200b\n모두 모두 카카오프렌즈 판교 아지트점에서 만나요-!\n\n\n카카오프렌즈 판교아지트점경기도 성남시 분당구 판교역로 166\n이 블로그의 체크인\n이 장소의 다른 글\n\u200b [...] 카카오프렌즈 판교 아지트점\n문구, 펜시\n\u200b\n주소 : 경기 성남시 분당구 판교역로 166 1층\n시간 : 월 - 금 10:00 - 21:00 / 토 - 일 12:00 - 20:00\n주차장 : 지하 주차장이 있어요!\n카카오아지트는 회차 20분 무료, 기본 30분에 1,000원 입니다!\n판교역 4번출구에서

### 문서 기반 문서 검색 도구: Retriever

우리의 데이터에 대해 조회를 수행할 retriever도 생성합니다.

**실습에 활용한 문서**

2024_프로야구_리그규정_요약.pdf

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


이 코드는 웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 OpenAI 임베딩을 사용하여 문서 검색 시스템을 구축합니다.

여기서는 PDF 문서를 `Chroma` DB 에 저장하고 조회하는 retriever 를 생성합니다.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_chroma import Chroma

# PDF 파일 로드. 파일의 경로 입력
loader = PDFPlumberLoader("data/2024_프로야구_리그규정_요약.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = Chroma.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

이 함수는 `retriever` 객체의 `invoke()` 를 사용하여 사용자의 질문에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용됩니다.


In [5]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("프로야구 연장전은 몇회까지 해?")

[Document(metadata={'CreationDate': "D:20240403201030-09'00'", 'Creator': 'nPDF (pdftk 1.41)', 'ModDate': 'D:20241228092714Z', 'Producer': '3-Heightsâ—¢ PDF Optimization Shell 6.3.1.5 (http://www.pdf-tools.com)', 'file_path': 'data/2024_프로야구_리그규정_요약.pdf', 'page': 6, 'source': 'data/2024_프로야구_리그규정_요약.pdf', 'total_pages': 18}, page_content='있다.\n제13조 경기 전 연습시간\n1. 홈구단의 타격연습(경기개시 90분 전까지)\n2. 원정구단의 준비운동 및 타격연습 60분간(경기개시 30분 전까지)\n3. 홈구단의 수비연습 10분간(경기개시 20분 전까지)\n4. 원정구단의 수비연습 10분간(경기개시 10분 전까지)\n① 경기관리인은 그라운드 사정 또는 기타 이유로 경기에 지\n장을 가져올 염려가 있다고 판단할 경우에는 연습시간을\n단축 또는 생략할 수도 있다. 이 경우 원정구단에게 3시간\n전에 통고하여야 한다.\n② 원정구단 연습 중에는 홈구단 선수는 페어지역에 들어가서\n는 안 된다.\n③ 관객이 입장한 후에는 선수는 그라운드에서 반드시 유니폼\n을 착용하고 모자를 써야 하며, 언더셔츠 차림으로 연습할\n수 없다.\n제14조 현역선수 등 등록\n1. 벤치에 들어가는 인원\n경기 중(타순표의 교환 후부터 경기종료까지) 벤치에 들어갈\n수 있는 인원의 한도는 다음과 같이 KBO에 등록승인을 필한\n자에 한한다.\n감독 1명\n코치 9명(불펜코치 포함)\n현역선수 28명(9월 1일 이후부터 5명 추가하여 33명)\n단, 경기출장은 26명(9월 1일 이후부터는 31명)으로 한다.\n매니저 1명\n트레이너 2명\n5'),
 Document(metadata={'C

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환할 수 있습니다.


`create_retriever_tool` 함수로 `retriever` 를 도구로 변환합니다.

In [6]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### Agent 가 사용할 도구 목록 정의

이제 두 가지를 모두 만들었으므로, Agent 가 사용할 도구 목록을 만들 수 있습니다.

`tools` 리스트는 `search`와 `retriever_tool`을 포함합니다. 

In [7]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

## Agent 생성

이제 도구를 정의했으니 에이전트를 생성할 수 있습니다. 

먼저, Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

**참고**
- 멀티턴 대화를 지원하지 않는다면 "chat_history" 를 제거해도 좋습니다.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

다음으로는 Tool Calling Agent 를 생성합니다.

In [9]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

마지막으로, 생성한 `agent` 를 실행하는 `AgentExecutor` 를 생성합니다.

**참고**

- `verbose=False` 로 설정하여 중간 단계 출력을 생략하였습니다.

In [10]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 에이전트 실행하기

이제 몇 가지 질의에 대해 에이전트를 실행할 수 있습니다!

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의입니다(이전 상호작용을 기억하지 않습니다).


`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. 이 예제에서는 `input` 키에 `hi!` 값을 할당한 딕셔너리를 인자로 전달하고 있습니다. 이는 일반적으로 AI 에이전트, 함수 실행기, 또는 명령 처리기 등의 객체에서 입력을 처리하기 위해 사용됩니다.


In [11]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [12]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`




Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`


[{'title': '프로야구 포스트시즌 경기 일정 및 결과 총정리 2024 - 만개의 축제', 'url': 'https://filmk.tistory.com/entry/%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EA%B2%BD%EA%B8%B0-%EC%9D%BC%EC%A0%95-%EB%B0%8F-%EA%B2%B0%EA%B3%BC-%EC%B4%9D%EC%A0%95%EB%A6%AC2024', 'content': '2024 한국 프로야구 플레이오프에서 승리한 팀은 KIA와 맞붙게 되며, 그 승부는 한국시리즈 진출을 결정짓는 중요한 대결이 될 것입니다. 각 팀이 전력을', 'score': 0.80698997}, {'title': 'KBO 플레이오프/2024년 - 나무위키', 'url': 'https://namu.wiki/w/KBO%20%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84/2024%EB%85%84', 'content': '... 년 한국시리즈가 마지막이다. 나눔 올스타 팀 중 LG 트윈스가 플레이오프 진출을, KIA 타이거즈가 한국시리즈 직행을 하면서 앞으로의 포스트시즌 경기 결과에 상관', 'score': 0.80625874}, {'title': '2024 KBO 한국 프로야구 포스트시즌 일정 순위 예매 (와일드카드 준 ...', 'ur

`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공합니다.


In [13]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "프로야구 연장전은 몇회까지 하는지 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: pdf_search
query: 프로야구 연장전 몇회
Log: 
Invoking: `pdf_search` with `{'query': '프로야구 연장전 몇회'}`




Invoking: `pdf_search` with `{'query': '프로야구 연장전 몇회'}`


있다.
제13조 경기 전 연습시간
1. 홈구단의 타격연습(경기개시 90분 전까지)
2. 원정구단의 준비운동 및 타격연습 60분간(경기개시 30분 전까지)
3. 홈구단의 수비연습 10분간(경기개시 20분 전까지)
4. 원정구단의 수비연습 10분간(경기개시 10분 전까지)
① 경기관리인은 그라운드 사정 또는 기타 이유로 경기에 지
장을 가져올 염려가 있다고 판단할 경우에는 연습시간을
단축 또는 생략할 수도 있다. 이 경우 원정구단에게 3시간
전에 통고하여야 한다.
② 원정구단 연습 중에는 홈구단 선수는 페어지역에 들어가서
는 안 된다.
③ 관객이 입장한 후에는 선수는 그라운드에서 반드시 유니폼
을 착용하고 모자를 써야 하며, 언더셔츠 차림으로 연습할
수 없다.
제14조 현역선수 등 등록
1. 벤치에 들어가는 인원
경기 중(타순표의 교환 후부터 경기종료까지) 벤치에 들어갈
수 있는 인원의 한도는 다음과 같이 KBO에 등록승인을 필한
자에 한한다.
감독 1명
코치 9명(불펜코치 포함)
현역선수 28명(9월 1일 이후부터 5명 추가하여 33명)
단, 경기출장은 26명(9월 1일 이후부터는 31명)으로 한다.
매니저 1명
트레이너 2명
5

일로부터 10일이 경과한 후 KBO 리그 또는 퓨처스리그 엔트
리에 등록된 경우 자동으로 치료·재활선수 명단에서 말소된
다. 치료·재활선수 명단은 현역선수 등록일수 인정이 적용되
지 않으며, 현역선수 등록 기간이 아닌 때에도 치료·재활선수
명단은 운영된다. (2022.9.27 신설)
5. 코치 겸 선수, 선수로 등록된 자는 베이스코치의 임무를 담당
할 수 없다.
6. 현역선수등록은 28명

## 이전 대화내용 기억하는 Agent

이전의 대화내용을 기억하기 위해서는 `RunnableWithMessageHistory` 를 사용하여 `AgentExecutor` 를 감싸줍니다.

`RunnableWithMessageHistory` 에 대한 자세한 내용은 아래 링크를 참고해 주세요.

**참고**
- [RunnableWithMessageHistory](https://wikidocs.net/254682)

In [16]:

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [17]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "프로야구 연장전은 몇회까지 하는지 정보를 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: pdf_search
query: 프로야구 연장전 몇회
Log: 
Invoking: `pdf_search` with `{'query': '프로야구 연장전 몇회'}`




Invoking: `pdf_search` with `{'query': '프로야구 연장전 몇회'}`


있다.
제13조 경기 전 연습시간
1. 홈구단의 타격연습(경기개시 90분 전까지)
2. 원정구단의 준비운동 및 타격연습 60분간(경기개시 30분 전까지)
3. 홈구단의 수비연습 10분간(경기개시 20분 전까지)
4. 원정구단의 수비연습 10분간(경기개시 10분 전까지)
① 경기관리인은 그라운드 사정 또는 기타 이유로 경기에 지
장을 가져올 염려가 있다고 판단할 경우에는 연습시간을
단축 또는 생략할 수도 있다. 이 경우 원정구단에게 3시간
전에 통고하여야 한다.
② 원정구단 연습 중에는 홈구단 선수는 페어지역에 들어가서
는 안 된다.
③ 관객이 입장한 후에는 선수는 그라운드에서 반드시 유니폼
을 착용하고 모자를 써야 하며, 언더셔츠 차림으로 연습할
수 없다.
제14조 현역선수 등 등록
1. 벤치에 들어가는 인원
경기 중(타순표의 교환 후부터 경기종료까지) 벤치에 들어갈
수 있는 인원의 한도는 다음과 같이 KBO에 등록승인을 필한
자에 한한다.
감독 1명
코치 9명(불펜코치 포함)
현역선수 28명(9월 1일 이후부터 5명 추가하여 33명)
단, 경기출장은 26명(9월 1일 이후부터는 31명)으로 한다.
매니저 1명
트레이너 2명
5

일로부터 10일이 경과한 후 KBO 리그 또는 퓨처스리그 엔트
리에 등록된 경우 자동으로 치료·재활선수 명단에서 말소된
다. 치료·재활선수 명단은 현역선수 등록일수 인정이 적용되
지 않으며, 현역선수 등록 기간이 아닌 때에도 치료·재활선수
명단은 운영된다. (2022.9.27 신설)
5. 코치 겸 선수, 선수로 등록된 자는 베이스코치의 임무를 담당
할 수 없다.
6. 현역선수등록은 28명

In [18]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
In professional baseball, extra innings can be played up to a maximum of 12 innings, while in the KBO postseason, it can be extended to 15 innings. If the game remains undecided, it is treated as a tie.

> Finished chain.
[최종 답변]
In professional baseball, extra innings can be played up to a maximum of 12 innings, while in the KBO postseason, it can be extended to 15 innings. If the game remains undecided, it is treated as a tie.


## Agent 템플릿

다음은 전체 템플릿 코드 입니다.


In [ ]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/2024_프로야구_리그규정_요약.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# Prompt 를 정의합니다 - 이 부분을 수정할 수 있습니다!
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의합니다 ##########

# 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "프로야구 연장전은 몇회까지 하는지 문서에서 찾아줘"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "2024년 프로야구 플레이오프 진출 5개팀을 검색해서 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 한국 시리즈 일정을 추가하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)